In [2]:
import pandas as pd
import numpy as np
import xlwings as xw
import os


store_soh = pd.read_excel("store_soh.xlsx")
store_sales = pd.read_excel("store_sales.xlsx")

#Filter Three cols: Article, Site, Total Stock qauntity then aggregate them over qty to remove duplicates.
store_soh_article_site = store_soh[['Article','Site','Total Stock Quantity']].groupby(['Article','Site'])['Total Stock Quantity'].sum().reset_index()

## To change the number of days of sales

# Set the filter condition
n_days = 60

#finding the last date in the store sales df
max_date = store_sales['Day of Billing Date'].max()
#filtering the last date n days before
filter_date = max_date - pd.Timedelta(days=n_days)
print('start date:',filter_date)
print('end date:',max_date)
# filtering  the df for those rows where data is greater than last date. 
filtered_store_sales = store_sales[store_sales['Day of Billing Date']> filter_date]
#grouping store sales to eliminate any duplicate instance of Article and Qty and aggregate by POS Billing
filtered_store_sales_sub = filtered_store_sales[['Article','Site','POS Billing Quantity','POS Gross Sales','POS Total Discount']].groupby(['Article','Site']).agg({'POS Billing Quantity':'sum','POS Gross Sales':'sum','POS Total Discount':'sum'}).reset_index()
filtered_store_sales_sub

# Now we want to make sure that all the combination of Article + Site are present in our consolidated data
# To do that we take article + Site from both the store stock and sales data and concatenate
# There will be overlaps between the two, for that we remove the duplicates

# concatenatiing
master_article_site = pd.concat([filtered_store_sales_sub[['Article','Site']],store_soh_article_site[['Article','Site']]])
                                                           
# removing duplicates
master_article_site_dd = master_article_site.drop_duplicates(subset=['Article','Site'])
master_article_site_dd                                                          


Build_slst_df = master_article_site_dd.copy() #building sales stock df by left merging both the dataframes
# But first we need to see if all the articles are present in the MCC sheet

mcc = pd.read_excel("mcc.xlsx")

# dropping  articles which are duplicates as we need to vlookup the attributes
mcc = mcc.drop_duplicates(subset='Article')

mcc.columns

master = mcc.copy()
master.loc[:,'Brand'] = mcc['Brand'].str.upper().copy()

#Checking if all the Articles in the sales stock are contained in the master file
if Build_slst_df['Article'].isin(master['Article']).all():
    a = "yes"
else:
    a = "No"

# Getting the rows in the master sheet for which the articles are not there in the Build...df
not_contained = ~Build_slst_df['Article'].isin(master['Article'])
rows_not_contained = Build_slst_df[not_contained]
print(rows_not_contained)

Build_slst_df.duplicated().sum()

ss_df_mcc = Build_slst_df.merge(master.drop_duplicates(subset='Article'),how='inner',on='Article')
ss_df_mcc

#Now vlooking the sales quantitites from sales data
ss_df_mcc_stsoh = ss_df_mcc.merge(filtered_store_sales_sub,how='left',on=['Article','Site'])
ss_df_mcc_stsoh

# vlooking the stock quantities from stock data
ss = ss_df_mcc_stsoh.merge(store_soh_article_site,how='left',on=['Article','Site']).fillna(0)
ss

ss.columns

ss.groupby([' C Technique','MRP Bin']).agg({'POS Billing Quantity':'sum','Total Stock Quantity':'sum',
                                           'POS Gross Sales':'sum','POS Total Discount':'sum'}).reset_index()


# this is a function to create a dataframe with all sites and all crafts
def salesstock_all(metric):
    consolidated_df = pd.DataFrame()

    for brand in ss['Brand'].unique():
        for site in ss['Site'].unique():

            #filtering the brand and the site from the final sales to stock data
            ss_site_brand = ss[(ss['Brand']==brand) & (ss['Site']==site)]

            #Aggregating the sales and stock dataa as per the metric ( eg. C-Tehnique)
            ss_final = ss_site_brand.groupby([' C Technique','MRP Bin']).agg({'POS Billing Quantity':'sum','Total Stock Quantity':'sum',
            'POS Gross Sales':'sum','POS Total Discount':'sum'}).reset_index()

            # Creating sales and stock contribution columns respectively
            ss_final["%sales_cont"] = (ss_final['POS Billing Quantity']*100/ss_final['POS Billing Quantity'].sum().round(10)).fillna(0)

            ss_final["%stock_cont"] = (ss_final['Total Stock Quantity']*100/ss_final['Total Stock Quantity'].sum().round(10)).fillna(0)
            ss_final["Sales/SOH"] = ((ss_final['%sales_cont'])/(ss_final['%stock_cont'])).fillna(0)
            ss_final["MD%"] = ((ss_final['POS Total Discount'])/(ss_final["POS Gross Sales"])).fillna(999)
            ss_final['rank_sales']=ss_final['Sales/SOH'].rank(ascending=False,method='min',na_option='bottom')
            ss_final['rank_md'] = ss_final['MD%'].rank(ascending=False,method='min',na_option='bottom')
            max_rank = ss_final['rank_md'].max()
            ss_final['rev_rank_md'] = max_rank - ss_final['rank_md']+1
            #set rank to the Lowest if the corresponding value in another column is zero


            #sorting the values by sales contribution
            ss_final = ss_final.sort_values(by='%sales_cont',ascending=False)

            # Add columns for brand and sites
            ss_final['Brand'] = brand
            ss_final['Site'] = site
            ss_final['resultant'] = (0.70*ss_final['rank_sales'])+(0.30*ss_final['rev_rank_md'])



            # create a mask to identify rows where the condition is not met
            mask = ss_final['MD%'] != 999

            # Assign ranks to the rows where the condition is met, and NaN to the rows where the condition is not met
            ss_final['frank'] = ss_final.loc[mask,'resultant'].rank(ascending=True,method='min',na_option='bottom')-1




            # Append the current DataFrame to the consolidated DataFrame
            consolidated_df = pd.concat([consolidated_df,ss_final[['Brand','Site',metric,'MRP Bin','POS Billing Quantity','Total Stock Quantity','POS Gross Sales','POS Total Discount',
            '%sales_cont','%stock_cont','Sales/SOH','MD%','rank_sales','rank_md','rev_rank_md','resultant','frank']]])



    
    return consolidated_df


# this is a function to generate a dataframe with all crafts and all sites.
ssa = salesstock_all(' C Technique')
ssa

average_temp = ssa[ssa['POS Billing Quantity']>0]
average_temp

# Calculate the average,maximum, and minimum values for POS Billing Quantity for each Site Name
grouped = average_temp.groupby(['Brand','Site'])['POS Billing Quantity'].agg(['mean','max','min'])

#Rename the columns for clarity
grouped.columns = ['Average Sales','Maximum Sales','Minimum Sales']

# Merge the grouped dataframe with the original dataframe based on 'Site Name'
average_temp = average_temp.merge(grouped,on=['Brand','Site'],how = 'left')

average_temp

# Create a new column 'num_rows' counting the number of rows for each Brand on Site Name
average_temp['num_rows']= average_temp.groupby(['Brand','Site'])['Brand'].transform('size')
average_temp

average_temp['adjust'] = average_temp['Average Sales'] + (average_temp['num_rows']-average_temp['frank'])+((average_temp['Maximum Sales']-average_temp['Minimum Sales'])/average_temp['num_rows'])

# Calculate the sum of 'adjust' for each site name
site_adjust_sum = average_temp.groupby(['Brand','Site'])['adjust'].transform('sum')

# Calculate the % contribution of 'adjust' for each row
average_temp['adjust_contribution'] = ((average_temp['adjust']/site_adjust_sum)*100)
average_temp

#average_temp['gap%'] = average_temp['adjust_contribution']-average_temp["%stock_cont"] if average_temp['adjust_contribution']<100 else 100
#average_temp

#import pandas as pd

# Assuming you have a DataFrame called average_temp with columns 'adjust_contribution' and '%stock_cont'

average_temp['gap%'] = average_temp.apply(lambda row: row['adjust_contribution'] - row['%stock_cont'] 
                                          if row['adjust_contribution'] < 100 else 100, axis=1)


average_temp

discap = pd.read_excel("discap.xlsx")
discap

discap = discap[discap['Display Capacity']>0]
discap

# Step 1 : Merge the DataFrames based on Brand and Site Name using an inner merge
merged_df = average_temp.merge(discap,on=["Brand","Site"],how="inner")

# Step 2: Assign the Display Capacity values to the corresponding rows in df1
average_temp['Display Capacity'] = merged_df['Display Capacity']

average_temp['gap%'] = average_temp['gap%'].apply(lambda x:0 if x<0 else x)
average_temp

# Step 1 : Merge the DataFrames based on Brand and Site Name using an inner merge
groupbydf = average_temp.groupby(["Brand","Site"])['Total Stock Quantity'].sum().reset_index()
groupbydf

# Step 1 : Merge the DataFrames based on Brand and Site Name using an inner merge
merged_df = average_temp.merge(groupbydf,on=["Brand","Site"],how="inner")
merged_df

# Step 2: Assign the Display Capacity values to the corresponding rows in df1
average_temp['BrandStock'] = merged_df['Total Stock Quantity_y']
average_temp['Balance'] = average_temp['Display Capacity']-average_temp['BrandStock']

# Calculate the total gap for each group (brand and site)
average_temp['total_gap'] = average_temp.groupby(['Brand', 'Site'])['gap%'].transform('sum')

# Calculate the normalized gap (normgap)
average_temp['normgap'] = average_temp['gap%']/ average_temp['total_gap']

average_temp["final_gap"] = (average_temp['normgap']*average_temp['Balance'])
average_temp["final_gap"] = average_temp["final_gap"].fillna(0).astype(int)
average_temp

xw.view(average_temp)

start date: 2023-01-31 00:00:00
end date: 2023-04-01 00:00:00
Empty DataFrame
Columns: [Article, Site]
Index: []
